# Tutorial 22: Data-driven corrections in PPE-ROM

In this tutorial, new data-driven corrections are introduced in the pressure Poisson formulation.
The case considered is the flux around a circular cylinder.

## Data collection: offline stage

The first step is to run the Openfoam simulation and to find the POD modes for velocity, supremizer and pressure fields. In this case we consider the supremizer formulation, so run on terminal: *offline 'poisson'*.

In this way the folder 'ITHACAoutput' is created. Snapshots data are collected in 'ITHACAoutput/Offline'. The matrices used to assemble the reduced systems are stored in 'ITHACAoutput/Matrices'. The POD modes for velocity and pressure are stored in 'ITHACAoutput/POD'.

## Definition of the PPE-ROM dynamical system: online stage

We import useful modules:

In [1]:
import numpy as np
import torch
from torch import nn
import matplotlib
import matplotlib.pyplot as plt
import os
from DD_PPE_ROM import PPE_ROM
from sklearn import *

The reference folder from which data is extracted is also defined.

In [3]:
folder = './ITHACAoutput'

Now the PPE_ROM class is used to define the different dynamical systems which should be solved and compared in the online phase.

We here choose a reduced number of velocity and pressure modes ($N_u$ and $N_p$, respectively) for the reduced model. The modes are denoted as: $(\boldsymbol{\phi_i})_{i=1}^{N_u}$ and $(\chi_i)_{i=1}^{N_p}$. $N_{sup}$ is set to 0 for the Poisson approach.

In [ ]:
Nu = 5
Nsup = 0
Np = 5
Nnut = 5
nu = 1e-4  #viscosity

The reduced approximated fields are computed at each time step by solving a dynamical system with unknowns $\mathbf{a}=(a_i)_{i=1}^{N_u}$ and $\mathbf{b}=(b_i)_{i=1}^{N_p}$, which are the vectors of coefficients for velocity and pressure. The reduced approximated fields are defined as:
\begin{equation}
\mathbf{u}_r = \sum_{i=1}^{N_u} a_i(t) \boldsymbol{\phi_i} \text{ and } \bar{p}_r = \sum_{i=1}^{N_u} b_i(t) \chi_i
\end{equation}

The standard PPE reduced system (without any correction term and without turbulence modelling), obtained with a POD-Galerkin approach, is the following:
\begin{equation}
    \begin{cases}
    \mathbf{M} \dot{\mathbf{a}}=\nu(\mathbf{B}+\mathbf{B_T})\mathbf{a}-\mathbf{a}^T \mathbf{C} \mathbf{a}-\mathbf{H}\mathbf{b}+\tau \left( \sum_{k=1}^{N_{\text{BC}}}(U_{\text{BC},k}\mathbf{D}^k-\mathbf{E}^k \mathbf{a})\right),\\
    \mathbf{D}\mathbf{b} + \mathbf{a}^T \mathbf{G} \mathbf{a} -\nu \mathbf{N} \mathbf{a} -\mathbf{L} =\mathbf{0}.
    \end{cases}
\end{equation}

Matrices appearing in the previous system are defined as:
\begin{equation}
\begin{split}
&(\mathbf{M})_{ij}=(\boldsymbol{\phi}_i,\boldsymbol{\phi}_j)_{L^2(\Omega)}, \quad (\mathbf{B})_{ij}=(\boldsymbol{\phi}_i,\nabla \cdot \nabla \boldsymbol{\phi}_j)_{L^2(\Omega)}, \\
&(\mathbf{B_T})_{ij}=(\boldsymbol{\phi}_i,\nabla \cdot (\nabla \boldsymbol{\phi}_j)^T)_{L^2(\Omega)},\quad (\mathbf{C})_{ijk}=(\boldsymbol{\phi}_i,\nabla \cdot (\boldsymbol{\phi}_j \otimes \boldsymbol{\phi}_k))_{L^2(\Omega)}, \quad (\mathbf{H})_{ij}=(\boldsymbol{\phi}_i,\nabla \chi_j)_{L^2(\Omega)}\,, \\
&(\mathbf{D})_{ij}=(\nabla \chi_i,\nabla \chi_j)_{L^2(\Omega)}, \quad 
(\mathbf{G})_{ijk}=(\nabla \chi_i,\nabla \cdot (\boldsymbol{\phi}_j \otimes \boldsymbol{\phi}_k))_{L^2(\Omega)}, \\ &(\mathbf{N})_{ij}=(\mathbf{n} \times \nabla \chi_i,\nabla \boldsymbol{\phi}_j)_{\Gamma}, \quad (\mathbf{L})_{ij}=(\chi_i,\mathbf{n} \cdot \boldsymbol{R}_t)_{\Gamma}\, .
\end{split}
\end{equation}

The term $\tau \left( \sum_{k=1}^{N_{\text{BC}}}(U_{\text{BC},k}\mathbf{D}^k-\mathbf{E}^k \mathbf{a})\right)$ is a penalization term used to enforce the Dirichlet boundary conditions at the reduced order level. In particular, $N_{\text{BC}}$ is the number of velocity boundary conditions on $k $ different parts of the Dirichlet boundary; $U_{\text{BC},k}$ is the velocity non-zero component at the $k$-th part of the Dirichlet boundary; $\tau$ is a penalization factor, matrices $\mathbf{E}^k$ and vectors $\mathbf{D}^k$ are defined as:
\begin{equation} (\mathbf{E}^k)_{ij}=(\boldsymbol{\phi}_i, \boldsymbol{\phi}_j)_{L^2(\Gamma_{D_k})}, \quad (\mathbf{D}^k)_{i}=(\boldsymbol{\phi}_i)_{\Gamma_{D_k}}, \text{ for all }k=1,...,N_{\text{BC}}.\end{equation}

The time derivative is computed with a second order backward time scheme.

This system is solved at each time step, lasting 0.004 seconds, for 501 time steps, for a global online computational time of 2 seconds.

In [ ]:
ROM_case_stand = PPE_ROM(folder, Nu, Nsup, Np, 0, 0)
ROM_case_stand.solveOnline_standard(0.004,501)

The projections of the full order fields onto the subspace generated by these modes are:

\begin{equation}
\mathbf{\bar{u}}_r = \sum_{i=1}^{N_u} a^{snap}_i \boldsymbol{\phi_i} \text{ and } \bar{p}_r = \sum_{i=1}^{N_u} b^{snap}_i \chi_i,
\end{equation}
where the matrices of the coefficients $a^{snap}_i$ and $b^{snap}_i$ are evaluated from the POD and stored in the snapshots matrices *snapU* and *snapP*.

*U_proj* and *P_proj* compute the reconstruction percentage errors, which are defined as the errors of the projected fields and the full order solutions and represent the best possible result that can be achieved.

In [ ]:
U_err_proj = ROM_case_stand.U_proj()
P_err_proj = ROM_case_stand.P_proj()

When extra correction terms are considered in the previous system, we obtain the following formulation:
\begin{equation}
\begin{cases}
    &\mathbf{M}\dot{\mathbf{a}} = \nu (\mathbf{B}+\mathbf{B_T}) \mathbf{a} - \mathbf{a}^T \mathbf{C} \mathbf{a}  - \mathbf{H} \mathbf{b}+\tau \left( \sum_{k=1}^{N_{\text{BC}}}(U_{\text{BC},k}\mathbf{D}^k-\mathbf{E}^k \mathbf{a})\right) + \boldsymbol{\tau}^u(\mathbf{a}) \, ,\\
    &\mathbf{D}\mathbf{b}+ \mathbf{a}^T \mathbf{G} \mathbf{a}  - \nu \mathbf{N} \mathbf{a}- \mathbf{L} + \boldsymbol{\tau}^p(\mathbf{a}, \mathbf{b})=\mathbf{0}, 
    \end{cases}
\end{equation}

In the first equation the term $\boldsymbol{\tau^u}(\mathbf{a}, \mathbf{b})$ is a velocity correction term and the term $\boldsymbol{\tau^p}(\mathbf{a}, \mathbf{b})$ is a pressure correction term introduced to improve the results in terms of pressure accuracy. In this tutorial the two terms are evaluated by solving a unique least squares problem. 

1. First of all, the exact correction term is found from available data, at each time step:

The exact term is:

\begin{equation}
    \boldsymbol{\tau}_{\text{tot}}^{\text{exact}}(t_j) = \left(\boldsymbol{\tau}_u^{\text{exact}}(t_j),\boldsymbol{\tau}_p^{\text{exact}}(t_j)\right) \, \forall j=1,...,M\,.
\end{equation}

The exact correction term corresponding to $\boldsymbol{\tau}_p$ includes the contribution of two different terms:
\begin{equation}
\begin{split}
 &\boldsymbol{\tau}_p^{\text{exact}}(t_j)= \boldsymbol{\tau}_D^{\text{exact}}(t_j)+\boldsymbol{\tau}_G^{\text{exact}}(t_j) =\\&= \left( \overline{\mathbf{D_d}\mathbf{b}_d^{\text{snap}}(t_j)}^q \right)-\mathbf{D} \mathbf{b}_q^{\text{snap}}(t_j)+ \overline{(\mathbf{a}_d^{\text{snap}}(t_j))^T \mathbf{G_d} \mathbf{a}_d^{\text{snap}}(t_j)}^r -(\mathbf{a}_r^{\text{snap}}(t_j))^T\mathbf{G} \mathbf{a}_r^{\text{snap}}(t_j)\, ,   
\end{split}
\end{equation}

where matrix $\mathbf{D_d}$ and tensor $\mathbf{G_d}$ are defined as follows:

\begin{equation}
\mathbf{D_d}_{ij}=(\nabla \chi_i,\nabla \chi_j), \quad \mathbf{G_d}_{ijk} = (\nabla \chi_i, \nabla \cdot (\boldsymbol{\phi}_j \otimes \boldsymbol{\phi}_k)), \, i,j,k = 1,\ldots ,d\, .
\end{equation}

2. Then, an ansatz is proposed to model the approximated correction term:

\begin{equation}
    \boldsymbol{\tau}_{\text{tot}}^{\text{ansatz}}(t_j)=\tilde{J}_A \mathbf{ab}^{\text{snap}} (t_j)+ (\mathbf{ab}^{\text{snap}} (t_j))^T \tilde{J}_B \mathbf{ab}^{\text{snap}} (t_j)\,,
\end{equation}

where matrices $\tilde{J}_A \in \mathbb{R}^{(r+q) \times (r+q)}$ and $\tilde{J}_B \in \mathbb{R}^{(r+q) \times (r+q) \times (r+q=}$ and vector $\mathbf{ab}^{\text{snap}} (t_j)=(\mathbf{a}_r^{\text{snap}} (t_j), \mathbf{b}_q^{\text{snap}} (t_j))$ $\in \mathbb{R}^{r+q}$. 
The unknowns computed by solving the optimization problem are $\tilde{J}_A$ and $\tilde{J}_B$.

The final correction which is inserted in the reduced system is divided in two vectors:
\begin{equation}
\tilde{J}_A \mathbf{ab} + \mathbf{ab}^T \tilde{J}_B \mathbf{ab}=\left( \boldsymbol{\tau}^u, \boldsymbol{\tau}^p\right) \text{ where }\boldsymbol{\tau}^u \in \mathbb{R}^{N_u}, \boldsymbol{\tau}^p \in \mathbb{R}^{N_p}\,.
\end{equation}

3. The matrices and tensors appearing in the ansatz are computed by solving an optimization problem, which is rewritten as a least squares problem between the exact and the approximated term. 

\begin{equation}
    \min_{\substack{\tilde{J}_A \in \mathbb{R}^{r_{\text{tot}} \times r_{\text{tot}}}; \\ \tilde{J}_B \in \mathbb{R}^{r_{\text{tot}} \times r_{\text{tot}} \times r_{\text{tot}}}}}{\sum_{j=1}^M || \boldsymbol{\tau}_{\text{tot}}^{\text{exact}}(t_j)-\boldsymbol{\tau}_{\text{tot}}^{\text{ansatz}}(t_j)||_{L^2(\Omega)}^2}\,.
\end{equation}

Passages to write the least squares problem are similar to tutorial DD-SUP-ROM_tutorial, but in this case the number of singular values $Ru$ for the matrix is the one that minimizes the error metric for pressure $\varepsilon_p(L^2)$, defined as:

\begin{equation}
    \varepsilon_p (L^2)= \sum_{j=1}^M ||p_{\text{sol}}(t_j)-p_r(t_j)||_{L^2(\Omega)} ,
\end{equation}

where $M$ is the number of time steps considered for the online simulation (2000 in our case, which corresponds to 8 seconds).

It is important to remark that the number of time steps used to compute the correction terms is 500 (2 seconds), whereas the online simulation lasts 5001 time steps (8 seconds). 

In [ ]:
Ru = 7
ROM_case = PPE_ROM(folder, Nu, Nsup, Np, 0, Ru)
ROM_case.solveOnline(0.004,501)

Now the formulation including the turbulence modelling is defined.
In this formulation, there is an extra-term depending on the eddy viscosity $\nu_t$, which is modelled as a linear combination of eddy viscosity modes $(\eta_i)_{i=1}^{N_{\nu_t}}$. The reduced eddy viscosity field is expressed as:

\begin{equation}
\nu_{t, r} = \sum_{i=1}^{N_{\nu_t}} g_i(t) \eta_i
\end{equation}

The reduced number of modes chosen for the eddy viscosity is the same of the velocity modes in our case ($N_{\nu_t}=N_u$). 

The reduced vector of coefficients $\mathbf{g}=(g_i)_{i=1}^{N_{\nu_t}}$ is computed at each time step with a regression technique starting from vector $\mathbf{a}$. 
In this case, $\mathbf{g}$ is computed making use of a feed-forward neural network, which is trained with the pairs of snapshots $(\mathbf{a}^{snap}, \mathbf{g}^{snap})$.

The reduced system written before can be rewritten in the following way:
When extra correction terms are considered in the previous system, we obtain the following formulation:
\begin{equation}
\begin{cases}
    &\mathbf{M}\dot{\mathbf{a}} = \nu (\mathbf{B}+\mathbf{B_T}) \mathbf{a} - \mathbf{a}^T \mathbf{C} \mathbf{a} + \mathbf{g}^T (\mathbf{C}_{\text{T1}}+\mathbf{C}_{\text{T2}}) \mathbf{a} - \mathbf{H} \mathbf{b}+\tau \left( \sum_{k=1}^{N_{\text{BC}}}(U_{\text{BC},k}\mathbf{D}^k-\mathbf{E}^k \mathbf{a})\right) + \boldsymbol{\tau}^u(\mathbf{a}) \, ,\\
    &\mathbf{D}\mathbf{b}+ \mathbf{a}^T \mathbf{G} \mathbf{a} -\mathbf{g}^T(\mathbf{C}_{\text{T3
    }} +\mathbf{C}_{\text{T4
    }})\mathbf{a} - \nu \mathbf{N} \mathbf{a}- \mathbf{L} + \boldsymbol{\tau}^p(\mathbf{a}, \mathbf{b})=\mathbf{0}, 
    \end{cases}
\end{equation}
where the new tensors are defined as follows.
\begin{equation}
    \begin{cases}
    &(\mathbf{C}_{\text{T1}})_{ijk}=(\boldsymbol{\phi}_i, \eta_j \nabla \cdot \nabla \boldsymbol{\phi}_k)_{L^2(\Omega)} \, ,\\
    &(\mathbf{C}_{\text{T2}})_{ijk}=(\boldsymbol{\phi}_i, \nabla \cdot \eta_j (\nabla \boldsymbol{\phi}_k)^T)_{L^2(\Omega)}\, , \\
    & (\mathbf{C}_{\text{T3}})_{ijk}=(\nabla \chi_i, \eta_j \nabla \cdot \nabla \boldsymbol{\phi}_k)_{L^2(\Omega)}\, , \\ &(\mathbf{C}_{\text{T4}})_{ijk}=(\nabla \chi_i, \nabla \cdot \eta_j(\nabla \boldsymbol{\phi}_k)^T)_{L^2(\Omega)}\, .
    \end{cases}
\end{equation}

The number of singular values retained in this case is $Ru\_T$.

In [ ]:
Ru_T = 7
ROM_caseT = SUP_ROM(folder, Nu,Nsup,Np,Nnut,Ru_T)
ROM_caseT.train_w([400,100],1500,500,1e-5, nn.ReLU) #training of neural network
ROM_caseT.solveOnlineT(0.004,501) #solve the dynamical system

The *errors* corresponding to each ROM case are here evaluated. This method calculates the percentage errors of the reduced velocity and pressure fields, at each time step, in the $L^2$ norm, with respect to the projected fields and the full order fields.

In [ ]:
ROM_cases = [ROM_case, ROM_caseT, ROM_case_stand]
for case in ROM_cases:
    case.errors()

Below the plots of the errors of the different cases with respect to the full order fields are represented, for both the velocity and the pressure fields.

In [ ]:
colors = ["royalblue","mediumorchid","crimson","lightseagreen","navy"]
markers = ["o","v","D","p","s"]
figvel2=plt.figure()
ax = figvel2.add_subplot(111)
plt.plot(ROM_case.errU_full, color=colors[0],marker=markers[0],markersize=5,markevery=50,label=r'Without turbulence term', fillstyle='none')
plt.plot(ROM_caseT.errU_full, color=colors[1],marker=markers[1],markersize=5,markevery=50,label=r'With turbulence term', fillstyle='none')
plt.plot(ROM_case_stand.errU_full, color=colors[2],marker=markers[2],markersize=5,markevery=50,label=r'Standard system', fillstyle='none')
plt.plot(U_err_proj, color=colors[3],marker=markers[3],markersize=5,markevery=50,label=r'Projection', fillstyle='none')
plt.ylabel(r'$\varepsilon_u^{full}$ [$\%$]')
ax.set_xticks([0,125,250,375,500])#,750,1000,1250,1500,1750,2000])
ax.set_xticklabels([0,0.5,1,1.5,2])#,3,4,5,6,7,8])
ax.set_xlabel('time [s]')
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.xlim(0,500)
plt.yscale("log")
plt.grid(True, which="both", ls="-")
plt.grid()
plt.show()
figvel2.savefig("error_U_PPE.pdf",bbox_inches='tight')

In [ ]:
figp=plt.figure()
ax = figp.add_subplot(111)
plt.plot(ROM_case.errP_full, color=colors[0],marker=markers[0],markersize=5,markevery=50,label=r'Without turbulence term', fillstyle='none')
plt.plot(ROM_caseT.errP_full, color=colors[1],marker=markers[1],markersize=5,markevery=50,label=r'With turbulence term', fillstyle='none')
plt.plot(ROM_case_stand.errP_full, color=colors[2],marker=markers[2],markersize=5,markevery=50,label=r'Standard system', fillstyle='none')
plt.plot(P_err_proj, color=colors[3],marker=markers[3],markersize=5,markevery=50,label=r'Projection', fillstyle='none')
plt.ylabel(r'$\varepsilon_p^{full}$ [$\%$]')
ax.set_xticks([0,125,250,375,500])#,750,1000,1250,1500,1750,2000])
ax.set_xticklabels([0,0.5,1,1.5,2])#,3,4,5,6,7,8])
ax.set_xlabel('time [s]')
plt.legend()
plt.xlim(0, 500)
plt.yscale("log")
plt.grid(True, which="both", ls="-")
plt.grid()
plt.show()
figp.savefig("error_P_PPE.pdf",bbox_inches='tight')